# Ejercicio 6

Utilice los scripts disponibles en la teoría y práctica para entrenar un combinador lineal. El modelo debe recibir tres dígitos binarios y retornar la representación decimal del número que resulta de la combinación de dígitos (para las entradas 000, 010 y 101 debe obtener las salidas 0, 2 y 5, respectivamente)

## Inciso a
Utilizando el bias o peso W0 (comportamiento normal)


In [4]:
# Entrenamos la neurona lineal para que aprenda la conversión de binario a decimal
from ClassNeuronaLineal import NeuronaLineal
import numpy as np


n=NeuronaLineal(0.3,700,1e-09, no_bias=False)

X = np.array([
  [0, 0, 0],
  [0, 0, 1],
  [0, 1, 0],
  [0, 1, 1],
  [1, 0, 0],
  [1, 0, 1],
  [1, 1, 0],
  [1, 1, 1]
])
y = [0, 1, 2, 3, 4, 5, 6, 7]

n.fit(X, y)

# Calcular el ECM final después del entrenamiento
y_pred = [n.predict(x) for x in X]
ecm_final = np.mean((np.array(y_pred) - np.array(y))**2)
print(f"ECM final: {ecm_final}")
print(f"Iteraciones realizadas: {n.iterations_}")
print(f"b_ (bias): {n.b_}")



ECM final: 7.0228685216729e-11
Iteraciones realizadas: 34
b_ (bias): 1.3894606895554654e-05


## Inciso b
Utilizando únicamente las tres entradas correspondientes a los dígitos binarios anulando el bias o W0 del cálculo.

In [3]:
n=NeuronaLineal(0.3,700,1e-09, no_bias=True)

n.fit(X, y)

# Calcular el ECM final después del entrenamiento
y_pred = [n.predict(x) for x in X]
ecm_final = np.mean((np.array(y_pred) - np.array(y))**2)
print(f"ECM final: {ecm_final}")
print(f"Iteraciones realizadas: {n.iterations_}")

ECM final: 2.5700127434593624e-12
Iteraciones realizadas: 18


## Inciso C

Compare la cantidad de iteraciones necesarias para obtener el vector de pesos correcto en ambos casos. Observe el valor del arco correspondiente al bias en a).

---

Se puede ver que converge mas rapido al no tener el bias. Esto se debe a que en el origen la recta pasa 0.

# Ejercicio 7

Utilizando los atributos numéricos del archivo `Autos.csv` entrene un combinador lineal para predecir el precio del auto en función del resto de los atributos. Tenga en cuenta la existencia de valores faltantes. Realice varias ejecuciones y observe los vectores de pesos obtenidos. ¿Puede identificar cuáles son los atributos con mayor incidencia en la determinación del precio? Explique.

In [5]:
import numpy as np
import pandas as pd
from ClassNeuronaLineal import NeuronaLineal

DATA_PATH='./Data/'
data=pd.read_csv(DATA_PATH+'autos.csv')
data.dtypes
data["price"] = pd.to_numeric(data["price"], errors="coerce")
numeric_data = data.select_dtypes(include=[np.number])
numeric_data.isnull().sum()
numeric_data = numeric_data.fillna(numeric_data.mean())

In [15]:
from sklearn.preprocessing import MinMaxScaler

X_raw = numeric_data.drop("price", axis=1)
T_raw = numeric_data["price"].values.reshape(-1, 1)

X_scaler, T_scaler = MinMaxScaler(), MinMaxScaler()
X = X_scaler.fit_transform(X_raw)
T = T_scaler.fit_transform(T_raw)


attributes = [col for col in numeric_data.columns if col != "price"]

n_runs = 10

weight_abs_list = []

for _ in range(n_runs):
    nl = NeuronaLineal(0.01, 1000, 1e-9, no_bias=False)
    nl.fit(X, T)
    weight_abs_list.append(np.abs(nl.w_))
    y_pred = [nl.predict(x) for x in X]
    ecm = np.mean((y_pred - T.flatten())**2)
    print(f"ECM: {ecm}")

weight_abs_array = np.array(weight_abs_list)
weight_mean = weight_abs_array.mean(axis=0)

weight_df = pd.DataFrame({
    "Attribute": attributes,
    "Mean weight (abs)": weight_mean
})

weight_df = weight_df.sort_values(by="Mean weight (abs)", ascending=False).reset_index(drop=True)
print(weight_df)

ECM: 0.06737348067518906
ECM: 0.06735113347875524
ECM: 0.06736654946842596
ECM: 0.06737131952978348
ECM: 0.06738281585290798
ECM: 0.06738347190704023
ECM: 0.06740015423944751
ECM: 0.06740123951897903
ECM: 0.06740688982647616
ECM: 0.06736653216903533
           Attribute  Mean weight (abs)
0        engine-size           0.710378
1           city-mpg           0.202016
2        curb-weight           0.127860
3  compression-ratio           0.075132
4             height           0.074770
5              width           0.054898
6          symboling           0.050343
7        highway-mpg           0.035495
8             length           0.027275
9         wheel-base           0.023189
